### 爬取步骤分析

在 Python 基础课中我们提过 编程思维（Computational Thinking）的概念，有如下四点：

- 问题分解：把现实生活中的复杂问题，逐步拆分成容易解决的小问题；
- 模式识别：根据已有的知识和经验，找出新问题和以前解决过的问题的相似性；
- 抽象思维：将问题里涉及的数据抽象到数据结构（变量、列表、字典等），把数据处理过程可重复执行部分抽象成函数；
- 算法设计：根据前三步的分析成果，设计步骤，写出算法，从而解决问题。

今天，我们就运用 编程思维 来写一个 自动发微博 的爬虫。

我们的目标是 发微博，发微博这件事可以拆成 登录微博 和 发送微博 两件事。接下来我们只需要解决 如何登录微博 和 如何发送微博 即可。将一个庞大且复杂的问题拆分成多个小问题，能让我们对问题有更清晰的认识，每解决一个小问题也会有成就感。将困难分摊开，也就没有那么难了。

根据已有的知识和经验，我们需要先成功登录微博，才能以当前账号信息发送微博。因此编写爬虫前，我们可以手动登录微博，记录下当前 cookie，并在爬虫程序内用 session 保留状态信息。

发送微博操作属于提交表单，一般都会用到 POST 请求，我们可以在开发者工具中的 Network 里看到。

我们也可以直接使用 selenium，但它爬取效率不高，且占资源，我们并不优先考虑它。

在写代码的过程中，我们需要思考用哪种数据结构存储数据更加合适，哪部分代码适合抽象成函数等，一点一点地解决每个小问题，最终完成整个爬虫。

现在，你应该对微博爬虫有了大体的认识。接下来，我们深入其中，看看爬虫具体要怎么写。

我们首先看一下如何登录微博。在这里偷偷告诉你一个小技巧：当我们爬一个网页时，可以先看看该网站有没有手机版。一般手机版的网页爬取数据比较容易，没有那么多的反爬虫限制。

In [ ]:
headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36',
  'cookie': '_T_WM=93695650748; WEIBOCN_FROM=1110005030; SUB=_2A25PyhaRDeRhGeBL7FUQ8S7Pwj2IHXVtNLrZrDV6PUJbktCOLWrCkW1NRsxyVUO8FVopeRv2h9uf6GCVTvPfWv1N; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9Wh4v8o11NG8buTpzWiFwCTy5NHD95QcSKMNeK27e0.pWs4Dqc_hi--fi-z7iKysi--NiK.4i-i2i--ciK.Ri-8si--Xi-zRi-8Wi--fi-z7iKysi--NiK.ci-8si--fi-82iK.7eK-Ne7tt; SSOLoginState=1657693890; MLOGIN=1; M_WEIBOCN_PARAMS=oid%3D4790777764645579%26lfid%3D102803%26luicode%3D20000174; XSRF-TOKEN=fb62db'
}

除了限制登录和 user-agent 之外，网站还会用一些其他的字段来反爬虫，判断是否为用户的正常访问。为了确保万无一失，我们手动浏览微博，尝试点赞、评论等操作，并逐个检查这些请求对应的 Request Headers（请求头，观察一下有哪些字段重复出现。这些字段很有可能就是我们需要伪装的重点。

### 发送微博

In [5]:
import requests

# 请求头中固定部分
headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36',
  'mweibo-pwa': '1',
  'x-requested-with': 'XMLHttpRequest',
  'cookie': '_T_WM=93695650748; WEIBOCN_FROM=1110005030; SUB=_2A25PyhaRDeRhGeBL7FUQ8S7Pwj2IHXVtNLrZrDV6PUJbktCOLWrCkW1NRsxyVUO8FVopeRv2h9uf6GCVTvPfWv1N; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9Wh4v8o11NG8buTpzWiFwCTy5NHD95QcSKMNeK27e0.pWs4Dqc_hi--fi-z7iKysi--NiK.4i-i2i--ciK.Ri-8si--Xi-zRi-8Wi--fi-z7iKysi--NiK.ci-8si--fi-82iK.7eK-Ne7tt; SSOLoginState=1657693890; MLOGIN=1; M_WEIBOCN_PARAMS=oid%3D4790777764645579%26lfid%3D102803%26luicode%3D20000174; XSRF-TOKEN=fb62db'
}

# 使用 session
session = requests.Session()
session.headers.update(headers)

# 发送微博所需请求头
compose_headers = {
  'origin': 'https://m.weibo.cn/',
  'referer': 'https://m.weibo.cn/compose/',
  'x-xsrf-token': 'af1140'
}

# 更新 headers
session.headers.update(compose_headers)

# 需要发送的微博信息
compose_data = {
  'content': '本条微博由 Python 发送',
  'st': 'af1140'
}
compose_req = session.post('https://m.weibo.cn/api/statuses/update', data=compose_data)
print(compose_req.status_code)
print(compose_req.json())
# 输出：403

403
{'ok': 0, 'errno': '100006', 'msg': '请求非法'}


In [3]:
import requests

# 请求头中固定部分
headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36',
  'mweibo-pwa': '1',
  'x-requested-with': 'XMLHttpRequest',
  'cookie': '_T_WM=93695650748; WEIBOCN_FROM=1110005030; SUB=_2A25PyhaRDeRhGeBL7FUQ8S7Pwj2IHXVtNLrZrDV6PUJbktCOLWrCkW1NRsxyVUO8FVopeRv2h9uf6GCVTvPfWv1N; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9Wh4v8o11NG8buTpzWiFwCTy5NHD95QcSKMNeK27e0.pWs4Dqc_hi--fi-z7iKysi--NiK.4i-i2i--ciK.Ri-8si--Xi-zRi-8Wi--fi-z7iKysi--NiK.ci-8si--fi-82iK.7eK-Ne7tt; SSOLoginState=1657693890; MLOGIN=1; M_WEIBOCN_PARAMS=oid%3D4790777764645579%26lfid%3D102803%26luicode%3D20000174; XSRF-TOKEN=fb62db'
}

# 使用 session
session = requests.Session()
session.headers.update(headers)

# 获取新 token 所需请求头
config_headers = {
  'origin': 'https://m.weibo.cn/',
  'referer': 'https://m.weibo.cn/'
}
session.headers.update(config_headers)

# 获取 token（x-xsrf-token 及 st 的值）
config_req = session.get('https://m.weibo.cn/api/config')
config = config_req.json()
st = config['data']['st']

# 发送微博所需请求头
compose_headers = {
  'origin': 'https://m.weibo.cn/',
  'referer': 'https://m.weibo.cn/compose/',
  'x-xsrf-token': st
}
session.headers.update(compose_headers)

# 需要发送的微博信息
compose_data = {
  'content': '本条微博由 Python 发送',
  'st': st
}
compose_req = session.post('https://m.weibo.cn/api/statuses/update', data=compose_data)
print(compose_req.json())
# 输出：{'ok': 1, 'data': 省略部分内容...}

{'ok': 1, 'data': {'visible': {'type': 0, 'list_id': 0}, 'created_at': 'Wed Jul 13 14:47:56 +0800 2022', 'id': '4790781901280534', 'mid': '4790781901280534', 'can_edit': False, 'show_additional_indication': 0, 'text': '本条微博由 Python 发送 ', 'textLength': 22, 'source': '微博 HTML5 版', 'favorited': False, 'pic_ids': [], 'is_paid': False, 'mblog_vip_type': 0, 'user': {'id': 6577110391, 'screen_name': '小白是个小疯子1573', 'profile_image_url': 'https://tvax3.sinaimg.cn/crop.0.0.996.996.180/007b6SRFly8fsf4a60sr5j30ro0romz4.jpg?KID=imgbed,tva&Expires=1657705677&ssig=gk6sXzgTf6', 'profile_url': 'https://m.weibo.cn/u/6577110391?uid=6577110391', 'statuses_count': 7, 'verified': False, 'verified_type': -1, 'close_blue_v': False, 'description': '', 'gender': 'm', 'mbtype': 0, 'urank': 4, 'mbrank': 0, 'follow_me': False, 'following': False, 'follow_count': 143, 'followers_count': '0', 'followers_count_str': '0', 'cover_image_phone': 'https://tva1.sinaimg.cn/crop.0.0.640.640.640/549d0121tw1egm1kjly3jj20hs0hsq4

In [ ]:
import requests

class WeiboSpider:
  def __init__(self):
    self.session = requests.Session()
    self.headers = {
      'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36',
      'mweibo-pwa': '1',
      'x-requested-with': 'XMLHttpRequest',
      'cookie': '_T_WM=93695650748; WEIBOCN_FROM=1110005030; SUB=_2A25PyhaRDeRhGeBL7FUQ8S7Pwj2IHXVtNLrZrDV6PUJbktCOLWrCkW1NRsxyVUO8FVopeRv2h9uf6GCVTvPfWv1N; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9Wh4v8o11NG8buTpzWiFwCTy5NHD95QcSKMNeK27e0.pWs4Dqc_hi--fi-z7iKysi--NiK.4i-i2i--ciK.Ri-8si--Xi-zRi-8Wi--fi-z7iKysi--NiK.ci-8si--fi-82iK.7eK-Ne7tt; SSOLoginState=1657693890; MLOGIN=1; M_WEIBOCN_PARAMS=oid%3D4790777764645579%26lfid%3D102803%26luicode%3D20000174; XSRF-TOKEN=fb62db'
    }
    self.session.headers.update(self.headers)

  def get_st(self):
    config_headers = {
      'origin': 'https://m.weibo.cn/',
      'referer': 'https://m.weibo.cn/'
    }
    self.session.headers.update(config_headers)

    config_req = self.session.get('https://m.weibo.cn/api/config')
    config = config_req.json()
    st = config['data']['st']
    return st

  def compose(self, content, st):
    compose_headers = {
      'origin': 'https://m.weibo.cn/',
      'referer': 'https://m.weibo.cn/compose/',
      'x-xsrf-token': st
    }
    self.session.headers.update(compose_headers)

    compose_data = {
      'content': content,
      'st': st
    }
    compose_req = self.session.post('https://m.weibo.cn/api/statuses/update', data=compose_data)
    print(compose_req.json())

  def send(self, content):
    st = self.get_st()
    self.compose(content, st)

weibo = WeiboSpider()
weibo.send('本条微博由 Python 发送')

可以看到，提示信息是 token校验失败。这个 token 又是什么东西？用来干嘛的？

且听我说，token 是令牌的意思，一般用于验证请求来源的可靠性，也属于反爬虫措施。那么这个请求哪里存放了令牌信息呢？

聪明的你肯定想到了请求头中的 x-xsrf-token，以及我们刚刚传入的 st 参数了吧！x-xsrf-token 的名字已经暗示了，它携带的就是 token 信息，而 st 参数的值总是和 x-xsrf-token 一致，所以携带的应该也是 token。

我们再发一条微博来验证一下，看看 st 有什么规律。

看到上面名称为 config 的请求了吗？每隔一段时间，微博会自动向 https://m.weibo.cn/api/config 地址发送 GET 请求，获取新 token 的值。我们之前的程序，可以说是拿着废弃令牌想通过微博这道“城门”。卫兵们自然不会放行了。